In [1]:
#importing the required packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
from scipy import stats
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

Using TensorFlow backend.


In [2]:
#reading the dataset
df = pd.read_csv(r"C:\Users\punit\Desktop\Thesis\New Dataset\Taking\punprecip.csv")
print(df.shape) 
df.head()
print(df.dtypes)
df

(10553, 10)
Date                  object
Longitude            float64
Latitude             float64
Elevation              int64
Max Temperature      float64
Min Temperature      float64
Precipitation        float64
Wind                 float64
Relative Humidity    float64
Solar                float64
dtype: object


,Date,Longitude,Latitude,Elevation,Max Temperature,Min Temperature,Precipitation,Wind,Relative Humidity,Solar
0,1/1/1985,75.9375,31.3790,260,11.4290,4.9620,4.2778,2.0940,0.8339,2.3622
1,1/2/1985,75.9375,31.3790,260,12.1760,3.5530,1.8917,1.0710,0.7806,1.6284
2,1/3/1985,75.9375,31.3790,260,15.2150,2.9080,0.0000,1.9556,0.6365,7.6308
3,1/4/1985,75.9375,31.3790,260,11.4850,7.6870,6.4167,1.5853,0.7293,1.9124
4,1/5/1985,75.9375,31.3790,260,9.8840,7.5810,25.3475,1.5331,0.9294,1.2978
5,1/6/1985,75.9375,31.3790,260,14.1030,1.6380,0.0721,2.1517,0.8337,12.6347
6,1/7/1985,75.9375,31.3790,260,13.3360,1.8250,0.2026,1.0486,0.7808,7.3375
7,1/8/1985,75.9375,31.3790,260,15.2420,4.8090,3.8590,1.8299,0.7434,12.2960
8,1/9/1985,75.9375,31.3790,260,15.6780,-2.5270,0.0000,1.1578,0.7619,12.8503
9,1/10/1985,75.9375,31.3790,260,16.5020,0.4880,0.0000,1.8768,0.7308,13.1645


In [3]:
# selecting the required columns
df = df[['Date', 'Precipitation']].copy()
df.head()

,Date,Precipitation
0,1/1/1985,4.2778
1,1/2/1985,1.8917
2,1/3/1985,0.0000
3,1/4/1985,6.4167
4,1/5/1985,25.3475


In [4]:
# convwerting Date into datetime format
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

Date             datetime64[ns]
Precipitation           float64
dtype: object

In [5]:
# indexing the Date column
df.head()
df.dtypes
df.set_index('Date', inplace=True)
df.head()

,Precipitation
Date,
1985-01-01,4.2778
1985-01-02,1.8917
1985-01-03,0.0000
1985-01-04,6.4167
1985-01-05,25.3475


In [6]:
# specifying the splits as 5
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

In [7]:
# performing the scaling and showing the 5 splits
dataset = df.Precipitation.values 
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
a = tscv.split(dataset)
for train_index, test_index in a:
    print("TRAIN:",train_index, "TEST:",test_index)
    train1, test1 = dataset[train_index], dataset[test_index]

TRAIN: [   0    1    2 ... 1760 1761 1762] TEST: [1763 1764 1765 ... 3518 3519 3520]
TRAIN: [   0    1    2 ... 3518 3519 3520] TEST: [3521 3522 3523 ... 5276 5277 5278]
TRAIN: [   0    1    2 ... 5276 5277 5278] TEST: [5279 5280 5281 ... 7034 7035 7036]
TRAIN: [   0    1    2 ... 7034 7035 7036] TEST: [7037 7038 7039 ... 8792 8793 8794]
TRAIN: [   0    1    2 ... 8792 8793 8794] TEST: [ 8795  8796  8797 ... 10550 10551 10552]


In [8]:
# converting an array of values 
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [9]:
# reshaping
look_back = 1
X_train, Y_train = create_dataset(train1, look_back)
X_test, Y_test = create_dataset(test1, look_back)

In [10]:
# reshaping the input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [11]:
# fitting the LSTM model
model = Sequential()
model.add(LSTM(128,activation="linear", input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, Y_train, epochs=100, batch_size=30, validation_data=(X_test, Y_test), 
                     verbose=2)
model.summary()

Train on 8793 samples, validate on 1756 samples
Epoch 1/100
 - 1s - loss: 0.0016 - val_loss: 0.0033
Epoch 2/100
 - 1s - loss: 0.0015 - val_loss: 0.0033
Epoch 3/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 4/100
 - 1s - loss: 0.0015 - val_loss: 0.0033
Epoch 5/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 6/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 7/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 8/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 9/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 10/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 11/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 12/100
 - 1s - loss: 0.0015 - val_loss: 0.0032
Epoch 13/100
 - 1s - loss: 0.0015 - val_loss: 0.0031
Epoch 14/100
 - 1s - loss: 0.0015 - val_loss: 0.0031
Epoch 15/100
 - 1s - loss: 0.0015 - val_loss: 0.0031
Epoch 16/100
 - 1s - loss: 0.0015 - val_loss: 0.0031
Epoch 17/100
 - 1s - loss: 0.0015 - val_loss: 0.0031
Epoch 18/100
 - 1s - loss: 0.0015 - val_loss: 0.0030
Epoch 1

In [12]:
# making the predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
# inverting the predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])

print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test[0], test_predict[:,0])))

Test Root Mean Squared Error: 14.77453020876196
